In [1]:
import chainer
import chainer.functions as F
import chainer.links as L
# trianerクラスが新機能として追加
from chainer import training
from chainer.training import extensions

In [4]:
# ネットワークの組み方・ハイパーパラ、メータの設定は変更なし
class MnistModel(chainer.Chain):
    def __init__(self):
        super(MnistModel,self).__init__(
                l1 = L.Linear(784,100),
                l2 = L.Linear(100,100),
                l3 = L.Linear(100,10))

    def __call__(self,x):    
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        return self.l3(h)

model = L.Classifier(MnistModel())
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [5]:
# mnistデータセットとってくる
train, test = chainer.datasets.get_mnist()

# ??
train_iter = chainer.iterators.SerialIterator(train, 100)
test_iter = chainer.iterators.SerialIterator(test, 100,repeat=False, shuffle=False)

In [8]:
updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (100, 'epoch'), out="result1")
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport( ['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))
trainer.extend(extensions.ProgressBar())

In [9]:
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
     total [#.................................................]  2.33%
this epoch [################..................................] 33.33%
       200 iter, 2 epoch / 100 epochs
    199.94 iters/sec. Estimated time to finish: 0:04:53.094769.
     total [#.................................................]  2.50%
this epoch [#########################.........................] 50.00%
       300 iter, 2 epoch / 100 epochs
     198.5 iters/sec. Estimated time to finish: 0:04:54.712171.
     total [#.................................................]  2.67%
this epoch [#################################.................] 66.67%
       400 iter, 2 epoch / 100 epochs
    198.57 iters/sec. Estimated time to finish: 0:04:54.106437.
     total [#.................................................]  2.83%
this epoch [#########################################.........] 83.33%
       500 iter, 2 epoch / 100 epochs
  

FileExistsError: [WinError 183] 既に存在するファイルを作成することはできません。: 'C:\\Users\\alluser\\Documents\\GitHub\\chainer_course\\result1\\logkumpyodd' -> 'result1\\log'